In [5]:
#i "nuget:https://api.nuget.org/v3/index.json" 

#r "nuget: Plotly.NET.Interactive"
#r "nuget: Plotly.NET.CSharp"

Restore sources https://api.nuget.org/v3/index.json Installed Packages Plotly.NET.CSharp, 0.13.0 Plotly.NET.Interactive, 5.0.0

In [6]:
// ...existing code...
using System;
using System.IO;
using System.Linq;
using System.Collections.Generic;

var folder = Path.Combine(Directory.GetCurrentDirectory(), "datas");
if (!Directory.Exists(folder))
    throw new Exception($"datas 文件夹未找到：{folder}");

var files = Directory.GetFiles(folder, "*.csv").OrderBy(f => f).ToArray();
if (files.Length == 0)
    throw new Exception("datas 目录中找不到 CSV 文件（期望第一列为索引，第二列为深度，后续列为测值）。");

var tracks = new List<(double[] depth, double[] value, string name)>();
double minD = double.MaxValue, maxD = double.MinValue;
foreach (var f in files)
{
    var lines = File.ReadAllLines(f).Where(l => !string.IsNullOrWhiteSpace(l)).ToArray();

    if (lines.Length == 0) continue;

    // 预解析确定列数与是否有 header（判断第二列第一行能否解析为 double）
    var firstParts = lines[0].Split(new[] { ',', ' ', '\t' }, StringSplitOptions.RemoveEmptyEntries);
    bool hasHeader = !double.TryParse(firstParts.ElementAtOrDefault(1) ?? "", out _);
    string[] header = null;
    int startLine = 0;
    if (hasHeader)
    {
        header = firstParts;
        startLine = 1;
    }

    // 找到最大列数
    int maxCols = lines.Select(l => l.Split(new[] { ',', ' ', '\t' }, StringSplitOptions.RemoveEmptyEntries).Length).Max();
    // 深度列是列索引1，测值列从索引2开始
    int valueCols = Math.Max(0, maxCols - 2);
    var depthListsPerValue = new List<List<double>>();
    var valListsPerValue = new List<List<double>>();
    for (int c = 0; c < valueCols; c++)
    {
        depthListsPerValue.Add(new List<double>());
        valListsPerValue.Add(new List<double>());
    }

    for (int i = startLine; i < lines.Length; i++)
    {
        var parts = lines[i].Split(new[] { ',', ' ', '\t' }, StringSplitOptions.RemoveEmptyEntries);
        if (parts.Length < 2) continue;
        if (!double.TryParse(parts[1], out double depth)) continue;

        // 对每个测值列尝试解析，缺失或无法解析则跳过该列的该行
        for (int c = 0; c < valueCols; c++)
        {
            int colIdx = 2 + c;
            if (colIdx >= parts.Length) continue;
            if (double.TryParse(parts[colIdx], out double v))
            {
                depthListsPerValue[c].Add(depth);
                valListsPerValue[c].Add(v);

                if (depth < minD) minD = depth;
                if (depth > maxD) maxD = depth;
            }
        }
    }

    // 为每个有效的测值列生成 track 项
    for (int c = 0; c < valueCols; c++)
    {
        var ds = depthListsPerValue[c];
        var vs = valListsPerValue[c];
        if (ds.Count > 1)
        {
            string colName = header != null && header.Length > 2 + c
                ? header[2 + c]
                : $"{Path.GetFileNameWithoutExtension(f)}_col{c+1}";
            tracks.Add((ds.ToArray(), vs.ToArray(), colName));
        }
    }
}

Console.WriteLine($"读取完成，找到 {tracks.Count} 条曲线，深度范围：{minD} - {maxD}");

读取完成，找到 150 条曲线，深度范围：504.0009155 - 528.9970328


In [7]:
// ...existing code...
// 2) 计算：构建公共深度栅格、插值并保存每条曲线的 vGrid（用于 heatmap）
if (tracks.Count == 0) throw new Exception("无可绘制曲线（tracks 为空）。");

double step = 0.1;
var grid = Enumerable.Range(0, (int)Math.Ceiling((maxD - minD) / step) + 1)
                     .Select(i => minD + i * step).ToArray();

// 线性插值
Func<double[], double[], double[], double[]> interp = (xs, ys, xsNew) =>
{
    var res = new double[xsNew.Length];
    for (int i = 0; i < xsNew.Length; i++)
    {
        double x = xsNew[i];
        if (x <= xs[0]) res[i] = ys[0];
        else if (x >= xs[^1]) res[i] = ys[^1];
        else
        {
            int idx = Array.BinarySearch(xs, x);
            if (idx >= 0) res[i] = ys[idx];
            else
            {
                int j = ~idx;
                double x0 = xs[j - 1], x1 = xs[j];
                double y0 = ys[j - 1], y1 = ys[j];
                res[i] = y0 + (y1 - y0) * (x - x0) / (x1 - x0);
            }
        }
    }
    return res;
};

double spacing = 1.2;
double horizontalScale = 0.45;
var xsList = new List<double[]>();
var ysList = new List<double[]>();
var names = new List<string>();

// 额外保存每条曲线在公共栅格上的原始值矩阵（未归一化），用于 heatmap
var vGridList = new List<double[]>();

foreach (var (dOrig, vOrig, name) in tracks)
{
    var d = (double[])dOrig.Clone();
    var v = (double[])vOrig.Clone();
    if (d[0] > d[^1]) { Array.Reverse(d); Array.Reverse(v); }

    var vGrid = interp(d, v, grid); // 原始测值插值到公共栅格
    vGridList.Add(vGrid);

    // 为了在同一行显示多条曲线，仍计算归一化后的横坐标用于其他可视化需要
    double vmin = vGrid.Min();
    double vmax = vGrid.Max();
    double mid = (vmin + vmax) / 2.0;
    double half = (vmax - vmin) / 2.0;
    if (half == 0) half = 1.0;

    var xs = vGrid.Select(val => (double)(xsList.Count * spacing) + ((val - mid) / half) * horizontalScale).ToArray();
    var ys = grid.Select(dd => -dd).ToArray();

    xsList.Add(xs);
    ysList.Add(ys);
    names.Add(name);
}

Console.WriteLine("计算完成，已生成 xs/ys/vGrid 列表与公共深度 grid。");
// ...existing code...

计算完成，已生成 xs/ys/vGrid 列表与公共深度 grid。


In [11]:
// 3) 图表构造与呈现（替换原第3单元）
using System;
using System.Collections.Generic;
using Plotly.NET;

// 检查必须的列表存在
if (xsList == null || ysList == null || names == null)
    throw new Exception("请先运行前两步（读取与计算），生成 xsList/ysList/names。");

var traces = new List<GenericChart>();
double spacing = 1.2; // 与计算步骤保持一致

// 绘制每条测井曲线
for (int i = 0; i < xsList.Count; i++)
{
    var xs = xsList[i];
    var ys = ysList[i];
    var nm = names[i];

    // 生成线图（显式指定泛型以避免类型推断问题）
    var trace = Plotly.NET.Chart2D.Chart.Line<double, double, string>(xs, ys, Name: nm);
    traces.Add(trace);
}

// 绘制每条轨道的中轴线（细灰线），范围取自 ysList（负深度）
double yMin = ysList.SelectMany(a => a).Min();
double yMax = ysList.SelectMany(a => a).Max();
for (int i = 0; i < xsList.Count; i++)
{
    double cx = i * spacing;
    var cxXs = new double[] { cx, cx };
    var cxYs = new double[] { yMin, yMax };
    var centerLine = Plotly.NET.Chart2D.Chart.Line<double, double, string>(cxXs, cxYs, Name: null);
    traces.Add(centerLine);
}

// 合并图层并设置尺寸与图例
var combined = Chart.Combine(traces)
                    .WithLegend(false)
                    .WithSize(1200, 600);

combined

<!-- Plotly chart will be drawn inside this DIV -->